In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# How to use the state evolution package

Let's look at a simple example of how to use the state evolution package with custom teacher-student covariance rmrices. The class has three components:
- `data_model`: this class defines everything concerning the generative model for data - i.e. it initialises the covariances $\Psi, \Phi, \Omega$ and the teacher weights $\theta_{0}$ and pre-computes all the quantities required for the state evolution.
- `model`: this class defines the task. It basically contains the updates for the overlaps and their conjugates. So far, we have implemented ridge and logistic regression.
- `algorithms`: this class defines the iterator for the state evolution.

In [2]:
from state_evolution.models.logistic_regression import LogisticRegression # logistic regression task
from state_evolution.algorithms.state_evolution import StateEvolution # Standard SP iteration

### Example 1: Custom data model: fixed sample complexity

Let's look at a simple example where we input the covariances.

In [3]:
from state_evolution.data_models.custom import Custom # Custom data model. You input the covariances

Recall that the Gaussian covariate model is defined by a teacher-student model with:
- Teacher : $y = f_{0}(\theta_{0}\cdot u)$, $\theta_{0}\sim\mathcal{N}(0,\rm{I}_{p})$
- Student : $\hat{y} = \hat{f}(w\cdot v)$
where $z\in\mathbb{R}^{p}$ and $v\in\mathbb{R}^{d}$ are jointly Gaussian variables with covariances
$$ \Psi = \mathbb{E}uu^{\top}\in\mathbb{R}^{p\times p}, \qquad \Phi = \mathbb{E}uv^{\top}\in\mathbb{R}^{p\times d}, \qquad \Omega = \mathbb{E}vv^{\top}\in\mathbb{R}^{v\times v}
$$.

The class `Custom` takes as input the three covariance matrices that define an instance of the model. 

As an example, let's look at a simple model of a Gaussian teacher $\theta_{0}\sim\mathcal{N}(0,\rm{I}_{p})$ and both the teacher and student are Random Feature models on Gaussian i.i.d. data, with different dimensions and activation functions:
$$
u = \rm{sign}\left(\frac{1}{\sqrt{D}}\bar{\rm{F}}c\right), \qquad v = \rm{erf}\left(\frac{1}{\sqrt{D}}\rm{F}c\right), \qquad c\sim\mathcal{N}(0,\rm{I}_{D})
$$

In this case recall that the covariances can be computed analytically, and are given by:

 \begin{align}
 \Psi = \bar{\kappa}_{1}^2 \bar{\rm{F}}\bar{\rm{F}}^{\top}+\bar{\kappa}_{\star}^2\rm{I}_{p}, && \Phi = \bar{\kappa}_{1}\kappa_{1} \bar{\rm{F}}\rm{F}^{\top}, && \Omega = \kappa_{1}^2 \rm{F}\rm{F}^{\top}+\kappa_{\star}^2\rm{I}_{d}
 \end{align}
 
with $\kappa_{1} \equiv \mathbb{E}\left[\xi\sigma(\xi)\right]$ and $\kappa_{\star}^2 \equiv \mathbb{E}\left[\sigma(\xi)\right]^2-\kappa_{1}^2$ for $\xi\sim\mathcal{N}(0,1)$ (idem for the bar). 

In [4]:
COEFICIENTS = {'relu': (1/np.sqrt(2*np.pi), 0.5, np.sqrt((np.pi-2)/(4*np.pi))), 
               'erf': (0, 2/np.sqrt(3*np.pi), 0.200364), 'tanh': (0, 0.605706, 0.165576),
               'sign': (0, np.sqrt(2/np.pi), np.sqrt(1-2/np.pi))}

In [5]:
D = 1000 # dimension of c
d = 2000 # dimension of x
p = 1000 # dimension of k

F_teacher = np.random.normal(0,1, (p,D)) / np.sqrt(D) # teacher random projection
F_student = np.random.normal(0,1, (d,D)) / np.sqrt(D) # student random projection

# Coefficients
_, kappa1_teacher, kappastar_teacher = COEFICIENTS['sign']
_, kappa1_student, kappastar_student = COEFICIENTS['erf']

# Covariances
Psi = (kappa1_teacher**2 * F_teacher @ F_teacher.T + kappastar_teacher**2 * np.identity(p))
Omega = (kappa1_student**2 * F_student @ F_student.T + kappastar_student**2 * np.identity(d))
Phi = kappa1_teacher * kappa1_student * F_teacher @ F_student.T 

# Teacher weights
theta = np.random.normal(0,1, p)

Now that we have our covariances, we can create our instance of `Custom`:

In [6]:
data_model = Custom(teacher_teacher_cov = Psi, 
                    student_student_cov = Omega, 
                    teacher_student_cov = Phi,
                    teacher_weights = theta)

Now, we need to load our task. Let's look at logistic regression. The `model` class takes as an input the sample complexity $\alpha = n/d$ and the $\ell_2$ regularisation $\lambda>0$

In [7]:
task = LogisticRegression(sample_complexity = 0.5,
                          regularisation= 0.01,
                          data_model = data_model)

All that is left is to initialise the saddle-point equation iterator:

In [8]:
sp = StateEvolution(model = task,
                    initialisation = 'uninformed',
                    tolerance = 1e-7,
                    damping = 0.5,
                    verbose = True,
                    max_steps = 1000)

Now, we can simply iterate it

In [9]:
sp.iterate()

t: 0, diff: 478.0045013006251, self overlaps: 0.048412094609408575, teacher-student overlap: 0.051274874532115336
t: 1, diff: 240.0002170073386, self overlaps: 0.15043222499943218, teacher-student overlap: 0.10913022451675058
t: 2, diff: 121.54556186933097, self overlaps: 0.3644023939495723, teacher-student overlap: 0.18324061599016236
t: 3, diff: 62.82752400014198, self overlaps: 0.7400197739562355, teacher-student overlap: 0.27438069344523214
t: 4, diff: 33.70151952262598, self overlaps: 1.2750247609456742, teacher-student overlap: 0.3754082520564553
t: 5, diff: 19.01417382856816, self overlaps: 1.9125382760019016, teacher-student overlap: 0.47590581496745876
t: 6, diff: 11.323818303388515, self overlaps: 2.584600622487235, teacher-student overlap: 0.5678099979836926
t: 7, diff: 7.084060451405307, self overlaps: 3.242597129663954, teacher-student overlap: 0.6473035545865558
t: 8, diff: 4.615815754692549, self overlaps: 3.858580282412054, teacher-student overlap: 0.7137041196847345
t:

Voila, now you can check the result with method `get_info`, which gives everything you might be interested in a dictionary.

In [10]:
sp.get_info()

{'hyperparameters': {'initialisation': 'uninformed',
  'damping': 0.5,
  'max_steps': 1000,
  'tolerance': 1e-07},
 'status': 1,
 'convergence_time': 62,
 'test_error': 0.38159910433570304,
 'train_loss': 0.10475628623477738,
 'overlaps': {'variance': 19.64921588846874,
  'self_overlap': 7.056042551862628,
  'teacher_student': 0.9552948607205072}}

### Example 2: Custom data model: whole learning curve

It is boring to repeat all the pipeline above every time you want to compute a new $\alpha$. Instead, we can encapsulate it in an `experiment` class which allows one to compute a whole learning curve in one go.

In [11]:
from state_evolution.experiments.learning_curve import CustomExperiment

The class `CustomExperiment` takes as argument the task you want (from those implemented), the regularisation and the data_model, apart from all the hyperparameters of the iterator.

In [12]:
my_experiment = CustomExperiment(task = 'logistic_regression', 
                                 regularisation = 0.01, 
                                 data_model = data_model, 
                                 initialisation='uninformed', 
                                 tolerance = 1e-7, 
                                 damping = 0.5, 
                                 verbose = True, 
                                 max_steps = 1000)

To compute the learning curve, you need to pass a python iterable with the values of the sample complexity you want to compute

In [13]:
my_experiment.learning_curve(alphas = [0.5, 1, 1.5])

Runninig sample complexity: 0.5
Runninig sample complexity: 1
Runninig sample complexity: 1.5


The method `.get_curve()` returns the learning curve as a `pandas.DataFrame`.

In [14]:
my_experiment.get_curve()

,task,gamma,lambda,rho,sample_complexity,V,m,q,test_error,train_loss
0,logistic_regression,0.5,0.01,0.979099,0.5,19.649216,0.955295,7.056043,0.381599,0.104756
1,logistic_regression,0.5,0.01,0.979099,1.0,10.289259,1.437530,9.174395,0.340768,0.156268
2,logistic_regression,0.5,0.01,0.979099,1.5,6.450970,1.619751,9.108898,0.317524,0.204456


Note you can save it in a csv, you can just call the method `save_experiment`

In [15]:
#my_experiment.save_experiment(name='testing')

### Example 3: defining a model directly as a function of the specta

Note that even though an instance of the Gaussian covariate model is defined by $(\Psi, \Phi, \Omega, \theta_{0})$, the saddle-point equations can be closed on the following scalar quantities:
\begin{align}
\rho = \frac{1}{p}\theta_{0}^{\top}\Psi\theta_{0}, && \omega_{i}\in \rm{spec}(\Omega), && t_{i} = \left(U^{\top}\Phi^{\top}\theta_{0}\theta_{0}^{\top}\Phi U\right)_{ii}, && i=1, \cdots, d
\end{align}
where $\rm{spec}(\Omega)$ are the eigenvalues of $\Omega$ and $U\in\mathbb{R}^{d\times d}$ are the eigenvectors of $\Omega$. 

Therefore, we can also define our `data_model` by directly passing these quantities to the class `CustomSpectra`

In [6]:
from state_evolution.data_models.custom import CustomSpectra

In [7]:
print('Computing the spectrum')
spec_Omega, U = np.linalg.eigh(Omega)

print('Projection in student basis')
t = np.diagonal(U.T @ Phi.T @ theta.reshape(p, 1) @ theta.reshape(1, p) @ Phi @ U)

print('Computing rho')
rho = 1/p * theta.dot(Psi @ theta)

Computing the spectrum
Projection in student basis
Computing rho


Note that $\rho\in\mathbb{R}$, but both $\{\omega_{i}\}_{i=1}^{d}$ and $\{t_{i}\}_{i=1}^{d}$ are $d$-dimensional quantities. Therefore, we will also need to pass $\gamma = p/d$ to our `data_model` in order to run the saddle-point equations.

In [9]:
data_model_spec = CustomSpectra(rho = rho, 
                                spec_Omega = spec_Omega, 
                                diagonal_term = t,
                                gamma = p/d)

In [13]:
my_experiment = CustomExperiment(task = 'logistic_regression', 
                                 regularisation = 0.01, 
                                 data_model = data_model_spec, 
                                 initialisation='uninformed', 
                                 tolerance = 1e-7, 
                                 damping = 0.5, 
                                 verbose = True, 
                                 max_steps = 1000)

my_experiment.learning_curve(alphas = [0.5])

Runninig sample complexity: 0.5
